## 减小尺寸

In [15]:
import torch

import torch.nn as nn
import torchvision
import torch.nn.functional as F
resnet50 = torchvision.models.resnet50(weights=True)

# 大步长卷积 
conv_stride = nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=3,
                        stride=2, 
                        padding=1)

# 空洞卷积
conv_dilation = nn.Conv2d(in_channels=1,
                        out_channels=1,
                        kernel_size=3,
                        dilation=2,
                        padding=2,
                        padding_mode='zeros')
x = torch.rand(1, 1, 8, 8)

out_stride = conv_stride(x)
out_dilation = conv_dilation(x)

print(out_stride.shape) # torch.Size([1, 1, 4, 4]) 
print(out_dilation.shape) # torch.Size([1, 1, 8, 8])

d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 8, 8])


In [16]:
torch.mean(x, dim=1, keepdim=True)

tensor([[[[0.4013, 0.2599, 0.3984, 0.6955, 0.5617, 0.7415, 0.3680, 0.3843],
          [0.9494, 0.5737, 0.1379, 0.1944, 0.9920, 0.4587, 0.6653, 0.0228],
          [0.1549, 0.4097, 0.5182, 0.9452, 0.3082, 0.3802, 0.5829, 0.3839],
          [0.7799, 0.2858, 0.5881, 0.4983, 0.5230, 0.8420, 0.7326, 0.2877],
          [0.8668, 0.0570, 0.2338, 0.4139, 0.5477, 0.5740, 0.3205, 0.1987],
          [0.0396, 0.8435, 0.0186, 0.4589, 0.0550, 0.9314, 0.5518, 0.9348],
          [0.7921, 0.9419, 0.6508, 0.3896, 0.9311, 0.4514, 0.9430, 0.3867],
          [0.3271, 0.5836, 0.0828, 0.7070, 0.8774, 0.7539, 0.8035, 0.1452]]]])

In [17]:
in_places=3
places=64
stride=1#不改变尺寸
expansion=4#通道倍增倍数
bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            # nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            # nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*expansion),
        )

In [18]:
x = torch.rand(1, 3, 256, 256)
bottleneck(x).shape

torch.Size([1, 256, 256, 256])

In [19]:
x=torch.rand(1, 3, 256, 256)
print(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=1, stride=1, padding=0, dilation=1)(x).shape)

torch.Size([1, 64, 256, 256])


In [20]:
dilation=1
kernel_size=5
kernel_ = dilation*(kernel_size-1) + 1
stride=2
padding=2

conv1=nn.Conv2d(in_channels=3, out_channels=64, kernel_size=kernel_size, stride=stride,padding=padding,dilation=dilation)
conv1(x).shape


torch.Size([1, 64, 128, 128])

In [29]:
inputsize=256

kernel_size=3

stride=1
padding=0
dilation=0
kernel_ = dilation*(kernel_size-1) + 1
(inputsize+2*padding-kernel_)/stride+1

256.0

In [ ]:
conv1_dila=nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2,padding=6,dilation=2)
conv1_dila(x).shape==conv1(x).shape

In [6]:
head=nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1))
conv_skip = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(64),
        )

In [7]:
class ResidualConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualConv, self).__init__()

        self.conv_block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(
                in_channels, out_channels, kernel_size=3, padding=1
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3,padding=1 ),
        )
        self.conv_skip = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
        )

    def forward(self, x):

        return self.conv_block(x) + self.conv_skip(x)
resc=ResidualConv(3,64)
resc(x).shape

torch.Size([1, 64, 256, 256])

In [5]:
head(x).shape

torch.Size([1, 64, 128, 128])

In [14]:
input_dim=3
output_dim=64
conv_block = nn.Sequential(
        nn.BatchNorm2d(input_dim),
        nn.ReLU(),
        nn.Conv2d(
            input_dim, output_dim, kernel_size=3, stride=1, padding=1
        ),
        nn.BatchNorm2d(output_dim),
        nn.ReLU(),
        nn.Conv2d(output_dim, output_dim, kernel_size=3, padding=1),
    )
conv_skip = nn.Sequential(
    nn.Conv2d(input_dim, output_dim, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(output_dim),
)
conv_block(x).shape

torch.Size([1, 64, 256, 256])

In [15]:
conv_skip(x).shape

torch.Size([1, 64, 256, 256])

In [12]:
import segmentation_models_pytorch as smp

d:\degree_thesis\tr2\py39torch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
input = torch.randn(3, 512, 64, 64)
# psa = PSA(channel=512, reduction=8)
# output = psa(input)
a = input.view(-1).sum()
a#.backward()

tensor(5760.3721)

In [18]:
# 常用网络结构
from torchvision.models.segmentation import fcn_resnet101
model = fcn_resnet101(pretrained=False,num_classes=3)

d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
resnet34 = torchvision.models.resnet34(pretrained=False)
filters = [64, 128, 256, 512]

firstlayer = nn.Sequential(*list(resnet34.children())[:3])
maxpool = list(resnet34.children())[3]
encoder1 = resnet34.layer1
encoder1

d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\degree_thesis\tr2\py39torch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, mome

In [5]:
from torchvision.models.resnet import BasicBlock
BasicBlock(3,64)

BasicBlock(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)